In [1]:
# HAVING
# 3번 이상 렌트한 유저 목록을 뽑는다

In [2]:
import pandas as pd
import numpy as np

import MySQLdb


db = MySQLdb.connect(
    "db.fastcamp.us",  # DATABASE_HOST
    "root",  # DATABASE_USERNAME
    "dkstncks",  # DATABASE_PASSWORD
    "sakila",  # DATABASE_NAME
    charset='utf8',
)

In [3]:
SQL_QUERY = """
    SELECT 
        rental.customer_id,
        COUNT(rental.rental_id)
    FROM 
        rental
    GROUP BY 1
    HAVING COUNT(rental.rental_id) >= 30
    ;
"""

pd.read_sql(SQL_QUERY, db)

,customer_id,COUNT(rental.rental_id)
0,1,32
1,5,38
2,7,33
3,15,32
4,20,30
5,21,35
6,23,30
7,26,34
8,27,31
9,28,32


In [4]:
# Subquery

SQL_QUERY = """
    SELECT 
        rental.customer_id,
        COUNT(rental.rental_id)
    FROM 
        rental
    GROUP BY 1
    HAVING COUNT(rental.rental_id) >= 30
"""

NEW_SQL_QUERY = """
    SELECT *
    FROM 
        customer
    JOIN ({SQL_QUERY}) AS many_rental_customer
    ON many_rental_customer.customer_id = customer.customer_id
""".format(SQL_QUERY=SQL_QUERY)

print(NEW_SQL_QUERY)


    SELECT *
    FROM 
        customer
    JOIN (
    SELECT 
        rental.customer_id,
        COUNT(rental.rental_id)
    FROM 
        rental
    GROUP BY 1
    HAVING COUNT(rental.rental_id) >= 30
) AS many_rental_customer
    ON many_rental_customer.customer_id = customer.customer_id



In [5]:
pd.read_sql(NEW_SQL_QUERY, db).head()

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update,customer_id,COUNT(rental.rental_id)
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20,1,32
1,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20,5,38
2,7,1,MARIA,MILLER,MARIA.MILLER@sakilacustomer.org,11,1,2006-02-14 22:04:36,2006-02-15 04:57:20,7,33
3,15,1,HELEN,HARRIS,HELEN.HARRIS@sakilacustomer.org,19,1,2006-02-14 22:04:36,2006-02-15 04:57:20,15,32
4,20,2,SHARON,ROBINSON,SHARON.ROBINSON@sakilacustomer.org,24,1,2006-02-14 22:04:36,2006-02-15 04:57:20,20,30


In [6]:
# Pandas Perspective

In [7]:
customer_df = pd.read_sql("SELECT * FROM customer;", db)
customer_df.head()

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20
2,3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,2006-02-14 22:04:36,2006-02-15 04:57:20
3,4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,2006-02-14 22:04:36,2006-02-15 04:57:20
4,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20


In [8]:
rental_df = pd.read_sql("SELECT * FROM rental", db)
rental_df.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [9]:
rentals_per_customer_df = rental_df.groupby('customer_id').agg({'customer_id': [np.size]})
rentals_per_customer_df.head()

,customer_id
,size
customer_id,
1,32
2,27
3,26
4,22
5,38


In [10]:
rentals_per_customer_df.columns = rentals_per_customer_df.columns.droplevel()
rentals_per_customer_df["customer_id"] = rentals_per_customer_df.index

In [11]:
merged_customer_df = pd.merge(customer_df, rentals_per_customer_df, on="customer_id")

In [12]:
merged_customer_df.rename(columns={
    "size": "Total Rental Count",        
}, inplace=True)

In [13]:
merged_customer_df

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update,Total Rental Count
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20,32
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20,27
2,3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,2006-02-14 22:04:36,2006-02-15 04:57:20,26
3,4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,2006-02-14 22:04:36,2006-02-15 04:57:20,22
4,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20,38
5,6,2,JENNIFER,DAVIS,JENNIFER.DAVIS@sakilacustomer.org,10,1,2006-02-14 22:04:36,2006-02-15 04:57:20,28
6,7,1,MARIA,MILLER,MARIA.MILLER@sakilacustomer.org,11,1,2006-02-14 22:04:36,2006-02-15 04:57:20,33
7,8,2,SUSAN,WILSON,SUSAN.WILSON@sakilacustomer.org,12,1,2006-02-14 22:04:36,2006-02-15 04:57:20,24
8,9,2,MARGARET,MOORE,MARGARET.MOORE@sakilacustomer.org,13,1,2006-02-14 22:04:36,2006-02-15 04:57:20,23
9,10,1,DOROTHY,TAYLOR,DOROTHY.TAYLOR@sakilacustomer.org,14,1,2006-02-14 22:04:36,2006-02-15 04:57:20,25


In [14]:
merged_customer_df[merged_customer_df["Total Rental Count"] >= 30]

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update,Total Rental Count
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20,32
4,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20,38
6,7,1,MARIA,MILLER,MARIA.MILLER@sakilacustomer.org,11,1,2006-02-14 22:04:36,2006-02-15 04:57:20,33
14,15,1,HELEN,HARRIS,HELEN.HARRIS@sakilacustomer.org,19,1,2006-02-14 22:04:36,2006-02-15 04:57:20,32
19,20,2,SHARON,ROBINSON,SHARON.ROBINSON@sakilacustomer.org,24,1,2006-02-14 22:04:36,2006-02-15 04:57:20,30
20,21,1,MICHELLE,CLARK,MICHELLE.CLARK@sakilacustomer.org,25,1,2006-02-14 22:04:36,2006-02-15 04:57:20,35
22,23,2,SARAH,LEWIS,SARAH.LEWIS@sakilacustomer.org,27,1,2006-02-14 22:04:36,2006-02-15 04:57:20,30
25,26,2,JESSICA,HALL,JESSICA.HALL@sakilacustomer.org,30,1,2006-02-14 22:04:36,2006-02-15 04:57:20,34
26,27,2,SHIRLEY,ALLEN,SHIRLEY.ALLEN@sakilacustomer.org,31,1,2006-02-14 22:04:36,2006-02-15 04:57:20,31
27,28,1,CYNTHIA,YOUNG,CYNTHIA.YOUNG@sakilacustomer.org,32,1,2006-02-14 22:04:36,2006-02-15 04:57:20,32
